In [10]:
# %pip install numpy

In [ ]:
import math
import numpy

def get_bracketing_indices(coords, target):
    if target <= coords[0]:
        return 0, 1
    elif target >= coords[-1]:
        return -2, -1
    else:
        for i in range(1, len(coords)):
            if coords[i - 1] <= target <= coords[i]:
                return i - 1, i
    raise ValueError("Target out of bounds")

def linear_interp(x0, x1, y0, y1, x):
    return y0 + (y1 - y0) * (x - x0) / (x1 - x0)

def log_interp(x0, x1, y0, y1, x):
    if x0 > 0 and x1 > 0 and x > 0:
        lx0, lx1, lx = math.log(x0), math.log(x1), math.log(x)
        return y0 + (y1 - y0) * (lx - lx0) / (lx1 - lx0)
    else:
        return linear_interp(x0, x1, y0, y1, x)

def polyfit_interp(xs, ys, x):
    # 최소제곱 2차 다항 보간
    n = len(xs)
    sum_x = sum(xs)
    sum_x2 = sum(xi**2 for xi in xs)
    sum_x3 = sum(xi**3 for xi in xs)
    sum_x4 = sum(xi**4 for xi in xs)
    sum_y = sum(ys)
    sum_xy = sum(xi * yi for xi, yi in zip(xs, ys))
    sum_x2y = sum((xi**2) * yi for xi, yi in zip(xs, ys))

    # 방정식 행렬 구성
    A = [
        [n, sum_x, sum_x2],
        [sum_x, sum_x2, sum_x3],
        [sum_x2, sum_x3, sum_x4]
    ]
    B = [sum_y, sum_xy, sum_x2y]

    # 가우스 소거법 (간단한 3x3 해법)
    from numpy.linalg import solve
    a0, a1, a2 = solve(A, B)
    return a0 + a1 * x + a2 * x ** 2

In [3]:
def linear_extrapolate_2d(x_coords, values_with_y, x_target, y_target):
    y_coords = [row[0] for row in values_with_y]

    def interp_y(col_idx):
        points = [(y_coords[i], values_with_y[i][col_idx]) for i in range(len(y_coords))]
        i0, i1 = get_bracketing_indices(y_coords, y_target)
        y0, v0 = points[i0]
        y1, v1 = points[i1]
        return linear_interp(y0, y1, v0, v1, y_target)

    i0, i1 = get_bracketing_indices(x_coords, x_target)
    x0, x1 = x_coords[i0], x_coords[i1]
    v0 = interp_y(i0)
    v1 = interp_y(i1)
    return linear_interp(x0, x1, v0, v1, x_target)


In [4]:
def log_extrapolate_2d(x_coords, values_with_y, x_target, y_target):
    y_coords = [row[0] for row in values_with_y]

    def interp_y(col_idx):
        points = [(y_coords[i], values_with_y[i][col_idx]) for i in range(len(y_coords))]
        i0, i1 = get_bracketing_indices(y_coords, y_target)
        y0, v0 = points[i0]
        y1, v1 = points[i1]
        return log_interp(y0, y1, v0, v1, y_target)

    i0, i1 = get_bracketing_indices(x_coords, x_target)
    x0, x1 = x_coords[i0], x_coords[i1]
    v0 = interp_y(i0)
    v1 = interp_y(i1)
    return log_interp(x0, x1, v0, v1, x_target)


In [5]:
def polyfit_extrapolate_2d(x_coords, values_with_y, x_target, y_target):
    y_coords = [row[0] for row in values_with_y]

    def interp_y(col_idx):
        y_vals = [row[col_idx] for row in values_with_y]
        return polyfit_interp(y_coords[-3:], y_vals[-3:], y_target)

    y_values = [interp_y(i) for i in range(len(x_coords))]
    return polyfit_interp(x_coords[-3:], y_values[-3:], x_target)


In [6]:
def compare_extrapolation_methods(x_coords, y_matrix, x_target, y_target):
    lin = linear_extrapolate_2d(x_coords, y_matrix, x_target, y_target)
    log = log_extrapolate_2d(x_coords, y_matrix, x_target, y_target)
    poly = polyfit_extrapolate_2d(x_coords, y_matrix, x_target, y_target)

    print(f"x = {x_target}, y = {y_target}")
    print(f"Linear   → {round(lin, 5)}")
    print(f"Log      → {round(log, 5)}")
    print(f"Polyfit  → {round(poly, 5)}")


In [7]:
wt = 1000 # weight of TNT equiv. in kgf (shoud be calculated based on R_dist & Pso1)
Wt = wt * 2.205 # weight of TNT equiv. in lbf (or shoud be calculated based on R_dist * 0.0032808 & PSO1)
 
# UFC 3-340-02 Figure 2-197
incident_OP = [0, 2, 4, 8, 16, 32]
rise_T = [
    [0.508005727, 3.659968582, 2.402535523, 1.569320331, 1.014967254, 0.645156521],
    [0.53821332, 3.723957592, 2.456675892, 1.596757521, 1.035272644, 0.654812742],
    [0.574349177, 3.770347843, 2.505824042, 1.624674409, 1.058602182, 0.667912897],
    [0.633609849, 3.807875783, 2.555955448, 1.661285927, 1.087831193, 0.684657255],
    [0.693956274, 3.807875783, 2.575012223, 1.69452154, 1.109594296, 0.698354478],
    [0.780065226, 3.742444802, 2.562291984, 1.724147684, 1.128993873, 0.708806927],
    [0.870550563, 3.623898318, 2.518263951, 1.741308888, 1.134598648, 0.712325727],
    [0.960372945, 3.483137251, 2.44454022, 1.73270704, 1.128993873, 0.70705405],
    [1.027816939, 3.356143416, 2.3788577, 1.715630609, 1.117867245, 0.700085787],
    [1.132229695, 3.146891076, 2.263923223, 1.673672196, 1.087831193, 0.681275133],
    [1.234705237, 2.950685419, 2.149213633, 1.608662678, 1.053372819, 0.656436106],
    [1.334839854, 2.753045781, 2.035270512, 1.534735557, 1.012457245, 0.627821766],
    [1.451455157, 2.537039705, 1.898946211, 1.439046259, 0.958780618, 0.591600108],
    [1.591992273, 2.292125469, 1.745625813, 1.322858057, 0.890141588, 0.546534288],
    [1.731073122, 2.075988045, 1.600716085, 1.21004372, 0.820300399, 0.49992543],
    [1.866065983, 1.880231436, 1.471474706, 1.115102765, 0.754069569, 0.455032448],
    [1.965641197, 1.74995344, 1.383150186, 1.050767833, 0.70705405, 0.423504034],
]
 


In [11]:
compare_extrapolation_methods(incident_OP, rise_T, x_target=10.8, y_target=2.71)

x = 10.8, y = 2.71
Linear   → 0.49486
Log      → 0.55086
Polyfit  → 0.59425


In [ ]:
# scaled_RT = round(linear_interpolate_2d_with_log_extrapolation(incident_OP, rise_T, 100, 4.2), 5)
# print("Scaled Rise Time:", scaled_RT)
# print("Rise Time:", round(scaled_RT * Wt**(1/3), 3), "ms")


NameError: name 'linear_interpolate_2d_with_log_extrapolation' is not defined